In [0]:
spark.read.csv(
  "s3://airport-intelligence/raw/Aeolus/Flight_Tab/flight_with_weather_2020.csv",
  header=True
).count()


4312091

In [0]:
df_raw = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("s3://airport-intelligence/raw/Aeolus/Flight_Tab/flight_with_weather_2020.csv")


In [0]:
df_raw.printSchema()
df_raw.count()
df_raw.show(5)


root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: double (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: timestamp (nullable = true)
 |-- DEP_TIME: timestamp (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: timestamp (nullable = true)
 |-- WHEELS_ON: timestamp (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: timestamp (nullable = true)
 |-- ARR_TIME: timestamp (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- FLIGHTS: double (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- ORIGIN_INDEX: integer (nullable = true)
 |-- DES

In [0]:
df_raw.write \
  .mode("overwrite") \
  .saveAsTable("flights_2020_raw")


In [0]:
%sql SHOW TABLES;


database,tableName,isTemporary
default,flights_2020_raw,false


In [0]:
%sql
SELECT COUNT(*) FROM flights_2020_raw;



COUNT(*)
4312091


In [0]:
df = spark.table("flights_2020_raw")


Data Cleaning


In [0]:
# Remove rows where all columns are NULL (no useful information)
df = df.dropna(how="all")


In [0]:
# Ensure flight date is valid and remove records with invalid dates
from pyspark.sql.functions import col, to_timestamp

df = df.withColumn("FL_DATE", to_timestamp(col("FL_DATE")))
df = df.filter(col("FL_DATE").isNotNull())


In [0]:
# Convert flight number from double to integer (identifier field)
df = df.withColumn(
    "OP_CARRIER_FL_NUM",
    col("OP_CARRIER_FL_NUM").cast("int")
)


In [0]:
# Create HH:MM formatted columns from timestamp fields for time analysis
from pyspark.sql.functions import date_format

time_cols = [
    "CRS_DEP_TIME",
    "DEP_TIME",
    "CRS_ARR_TIME",
    "ARR_TIME",
    "WHEELS_OFF",
    "WHEELS_ON"
]

for c in time_cols:
    df = df.withColumn(f"{c}_HHMM", date_format(col(c), "HH:mm"))


In [0]:
# Remove records with missing critical delay and taxi time values
critical_cols = ["DEP_DELAY", "ARR_DELAY", "TAXI_OUT", "TAXI_IN"]
df = df.dropna(subset=critical_cols)


In [0]:
# Remove records with missing origin or destination weather data
weather_cols = [
    "O_TEMP", "O_PRCP", "O_WSPD",
    "D_TEMP", "D_PRCP", "D_WSPD"
]

df = df.dropna(subset=weather_cols)


In [0]:
# Remove records that violate basic flight operation logic

df = df.filter(
    (col("TAXI_OUT") >= 0) &
    (col("TAXI_IN") >= 0) &
    (col("AIR_TIME") > 0) &
    (col("ACTUAL_ELAPSED_TIME") >= col("AIR_TIME"))
)


In [0]:
# Remove duplicate flight records using business key
df = df.dropDuplicates([
    "FL_DATE",
    "OP_CARRIER",
    "OP_CARRIER_FL_NUM",
    "ORIGIN",
    "DEST",
    "CRS_DEP_TIME"
])


In [0]:
# Standardize all column names to lower_snake_case format
for c in df.columns:
    df = df.withColumnRenamed(c, c.lower())


In [0]:
# Repartition dataset to improve Spark performance and memory usage
df = df.repartition(200)



In [0]:
# Final validation to check remaining NULL values across columns
from pyspark.sql.functions import sum

df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
]).show(truncate=False)


+-------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+-------+-----+------------+-----------+------------+----------+------+------+------+------+------+------+----------+-----------+----------+-----------+-----------------+-------------+-----------------+-------------+---------------+--------------+
|fl_date|op_carrier|op_carrier_fl_num|origin|dest|crs_dep_time|dep_time|dep_delay|taxi_out|wheels_off|wheels_on|taxi_in|crs_arr_time|arr_time|arr_delay|crs_elapsed_time|actual_elapsed_time|air_time|flights|month|day_of_month|day_of_week|origin_index|dest_index|o_temp|o_prcp|o_wspd|d_temp|d_prcp|d_wspd|o_latitude|o_longitude|d_latitude|d_longitude|crs_dep_time_hhmm|dep_time_hhmm|crs_arr_time_hhmm|arr_time_hhmm|wheels_off_hhmm|wheels_on_hhmm|
+-------+----------+-----------------+------+----+------------+--------+---------+--------+----------+--------

In [0]:
df.printSchema()

root
 |-- fl_date: timestamp (nullable = true)
 |-- op_carrier: string (nullable = true)
 |-- op_carrier_fl_num: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- crs_dep_time: timestamp (nullable = true)
 |-- dep_time: timestamp (nullable = true)
 |-- dep_delay: double (nullable = true)
 |-- taxi_out: double (nullable = true)
 |-- wheels_off: timestamp (nullable = true)
 |-- wheels_on: timestamp (nullable = true)
 |-- taxi_in: double (nullable = true)
 |-- crs_arr_time: timestamp (nullable = true)
 |-- arr_time: timestamp (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- crs_elapsed_time: double (nullable = true)
 |-- actual_elapsed_time: double (nullable = true)
 |-- air_time: double (nullable = true)
 |-- flights: double (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- origin_index: integer (nullable = true)
 |-- de

In [0]:
# Save cleaned dataset as curated table for downstream analytics
df.write \
  .mode("overwrite") \
  .saveAsTable("flights_2020_clean")
